In [1]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

nInst = 50
currentPos = np.zeros(nInst)

def zscore(series):
    return (series - series.mean()) / np.std(series)

def pairs_trading_strategy(prices_A, prices_B, window=50, z_threshold=1.0):
    # Calculate spread and its z-score
    spread = prices_A - prices_B
    z_scores = zscore(spread[-window:])
    current_z_score = z_scores[-1]

    # Trading logic
    if current_z_score > z_threshold:
        return -1, 1  # short A, long B
    elif current_z_score < -z_threshold:
        return 1, -1  # long A, short B
    else:
        return 0, 0  # No position

def mean_reversion_strategy(price_history, window=50, z_threshold=1.0):
    mean = price_history[-window:].mean()
    std = price_history[-window:].std()
    z_score = (price_history[-1] - mean) / std

    if z_score > z_threshold:
        return -1  # Short position
    elif z_score < -z_threshold:
        return 1   # Long position
    else:
        return 0   # No position

def random_forest_predict(prices, n_features=10):
    X = [prices[i:i+n_features] for i in range(len(prices)-n_features)]
    y = prices[n_features:]
    model = RandomForestRegressor()
    model.fit(X, y)
    return model.predict([prices[-n_features:]])[0]

def getMyPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if nt < 51:  # Need at least 51 data points to compute 50-day mean reversion
        return np.zeros(nins)
    
    # Assume asset 0 and asset 1 are our correlated pair (this is just an example)
    position_A, position_B = pairs_trading_strategy(prcSoFar[0], prcSoFar[1])
    currentPos[0] = position_A
    currentPos[1] = position_B

    # Apply mean reversion strategy for the remaining assets
    for inst in range(2, nins):
        price_history = prcSoFar[inst, :]
        currentPos[inst] = mean_reversion_strategy(price_history)
        
    # Apply Random Forest prediction for asset 0
    next_price = random_forest_predict(prcSoFar[0])
    if next_price > prcSoFar[0, -1]:
        currentPos[0] = 1
    else:
        currentPos[0] = -1
        
    return currentPos


In [2]:
import numpy as np
import pandas as pd
# from teamName import getMyPosition as getPosition

nInst = 0
nt = 0
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    #df=pd.read_csv(fn, sep='\s+', names=cols, header=None, index_col=0)
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.values.shape
    return (df.values).T

pricesFile="./prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

currentPos = np.zeros(nInst)

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(1,251): 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getMyPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] #prcHist[:,t-1]
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        clipPos = np.clip(newPosOrig, -posLimits, posLimits)
        newPos = np.array([np.trunc(x) for x in clipPos])
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)



(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print ("=====")
print ("mean(PL): %.1lf" % meanpl)
print ("return: %.5lf" % ret)
print ("StdDev(PL): %.2lf" % plstd)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)
print ("Score: %.2lf" % score)

Loaded 50 instruments for 250 days
Day 1 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 2 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 3 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 4 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 5 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 6 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 7 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 8 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 9 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 10 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 11 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 12 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 13 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 14 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 15 value: 0.00 todayPL: $0.00 $-traded: 0 return: 0.00000
Day 16 value: 0.00 todayPL: $0.00 $-traded: 

Day 125 value: -69.25 todayPL: $-1.44 $-traded: 24091 return: -0.00287
Day 126 value: -71.25 todayPL: $-2.00 $-traded: 24332 return: -0.00293
Day 127 value: -73.38 todayPL: $-2.12 $-traded: 24506 return: -0.00299
Day 128 value: -73.37 todayPL: $0.01 $-traded: 24698 return: -0.00297
Day 129 value: -72.04 todayPL: $1.33 $-traded: 24866 return: -0.00290
Day 130 value: -73.61 todayPL: $-1.57 $-traded: 25109 return: -0.00293
Day 131 value: -77.92 todayPL: $-4.31 $-traded: 25298 return: -0.00308
Day 132 value: -80.05 todayPL: $-2.13 $-traded: 25466 return: -0.00314
Day 133 value: -81.57 todayPL: $-1.53 $-traded: 25623 return: -0.00318
Day 134 value: -83.89 todayPL: $-2.32 $-traded: 25793 return: -0.00325
Day 135 value: -84.15 todayPL: $-0.26 $-traded: 26050 return: -0.00323
Day 136 value: -85.95 todayPL: $-1.80 $-traded: 26363 return: -0.00326
Day 137 value: -87.76 todayPL: $-1.81 $-traded: 26482 return: -0.00331
Day 138 value: -92.47 todayPL: $-4.71 $-traded: 26622 return: -0.00347
Day 139 

Day 241 value: -145.93 todayPL: $-0.68 $-traded: 59759 return: -0.00244
Day 242 value: -147.00 todayPL: $-1.07 $-traded: 60048 return: -0.00245
Day 243 value: -147.54 todayPL: $-0.54 $-traded: 60349 return: -0.00244
Day 244 value: -148.41 todayPL: $-0.87 $-traded: 60492 return: -0.00245
Day 245 value: -147.80 todayPL: $0.61 $-traded: 60730 return: -0.00243
Day 246 value: -149.72 todayPL: $-1.92 $-traded: 60891 return: -0.00246
Day 247 value: -148.46 todayPL: $1.26 $-traded: 61270 return: -0.00242
Day 248 value: -148.44 todayPL: $0.02 $-traded: 61605 return: -0.00241
Day 249 value: -148.39 todayPL: $0.05 $-traded: 62030 return: -0.00239
Day 250 value: -148.83 todayPL: $-0.43 $-traded: 62245 return: -0.00239
=====
mean(PL): -0.6
return: -0.00239
StdDev(PL): 1.48
annSharpe(PL): -6.37 
totDvolume: 62245 
Score: -0.74
